In [10]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START , END

from typing import Annotated
from langgraph.graph.message import add_messages


In [11]:
class Message(TypedDict):
    developer: str
    points: int

In [12]:
class Message2(TypedDict):
    key: str
    Title: str
    Description: str

In [13]:
from typing import TypedDict, List
def add_messages(current: List[Message], new: List[Message]) -> List[Message]:
    return current + new

class State(TypedDict):
    messages: Annotated[List[Message], add_messages]

class State2(TypedDict):
    messages2: Annotated[List[Message2], add_messages]    

state: State = {"messages": []}   
state2: State2 = {"messages2": []}  


In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")


In [28]:
'''Get all open issues'''
import requests

#==== Configuration ====
JIRA_BASE_URL = "https://ca-il-jira.il.cyber-ark.com:8443"
API_ENDPOINT = f"{JIRA_BASE_URL}/rest/api/2/search"
BEARER_TOKEN = "xGkGLtKttyM5Gnr57XLyPYMg2k1RwpnTyBiR8P"

# JQL query
JQL_QUERY = (
    'project = "Cross RND Ticket" AND type = Ticket '
    'AND component = Identity-Integrations AND status = "Investigating" '
 #   'AND (assignee is EMPTY or assignee = "Ratan sharma") ORDER BY Severity'
)

# Request parameters
params = {
    "jql": JQL_QUERY,
    "fields": "summary,description,customfield_19321,customfield_16122",
    "maxResults": 50
}

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Accept": "application/json"
}

# API call
response = requests.get(API_ENDPOINT, headers=headers, params=params)

# Store results in a list of lists
issue_list = [] 
print(response.status_code)
if response.status_code == 200:
    data = response.json()
    for issue in data.get("issues", []):
        key = issue.get("key")
        fields = issue.get("fields", {})

        title = fields.get("customfield_19321", {}).get("value", "No Title")
        description = fields.get("summary", "")
        description_clean = " ".join(description.split()) if description else "No Description"

        issue_list.append([key, title, description_clean])
        print(f"{key} | {title} | {description_clean}")  # Each on new line
else:
    print(f"Error {response.status_code}: {response.text}")
a =10
issue_prompt2 = "".join(
    f"Key: {inner[0]}, Title: {inner[1]}, Description: {inner[2]}\n"
    for inner in issue_list
)


200
CRT-9208 | CREATION FAILURE | failure in tenant creation due to timeout
CRT-9190 | CREATION FAILURE | Failed to create tenant


In [1]:
'''get developersa and their story points from current sprint'''
import requests
from requests.auth import HTTPBasicAuth
import json

# Configuration
print("amer copyright")
JIRA_BASE_URL = "https://ca-il-jira.il.cyber-ark.com:8443"
SPRINT_ID = "36339"
BEARER_TOKEN = "xGkGLtKttyM5Gnr57XLyPYMg2k1RwpnTyBiR8P"  # same token used in Bruno

url = f"{JIRA_BASE_URL}/rest/agile/1.0/sprint/{SPRINT_ID}/issue"

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)

if response.status_code != 200:
    print(f"Failed to fetch issues: {response.status_code}")
    exit()

data = response.json()

# Mapping: Developer -> Total Story Points
developer_points = {}

for issue in data.get("issues", []):
    fields = issue.get("fields", {})
    
    # Check if issue is not resolved/closed
    status = fields.get("status", {}).get("name", "").lower()
    if status in ["done", "completed", "closed"]:
        continue  # Skip non-active stories

    # Assignee info
    assignee = fields.get("assignee", {})
    name = assignee.get("name", "Unassigned")

    # Story points field
    story_points = fields.get("customfield_11820", 0)

    if story_points is None:
        story_points = 0

    # Accumulate
    developer_points[name] = developer_points.get(name, 0) + story_points

# Output
print("Developer-wise Active Story Points:")
for dev, points in developer_points.items():
    new_message: Message = {"developer": dev, "points": points}
    state["messages"] = add_messages(state["messages"], [new_message])
    print(f"{dev}: {points}")

amer copyright
Developer-wise Active Story Points:


In [17]:
'''pass tthe data to llm and get developer name for each CRTs'''
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv()
state_prompt = "".join(
    f"Developer: {msg['developer']}, Points: {msg['points']}\n"
    for msg in state["messages"]
)

llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"), 
    openai_api_base="https://openrouter.ai/api/v1",  
    model="qwen/qwq-32b:free"  
)

'''llm.bind_tools([assign_jira_issue,])'''
prompt = (
    state_prompt + issue_prompt2 +
    "You are an assignment engine. I have given you available developers with their story points. " +
    "Assign each CRT number to the developer with the fewest story points. " +
    "In case of a tie, assign arbitrarily. " +
    "Output must be ONLY in the format:\n\nCRT-xxxx: developer_name\nCRT-yyyy: developer_name\n\n" +
    "No explanation. No <think>. No comments. Only plain text mapping."
)
response = llm.invoke(state_prompt+ issue_prompt2 + prompt)
response_text = response.content.strip()
assignments = re.findall(r"CRT-\d+:\s+\w+", response_text)

result_list = []

for line in response_text.splitlines():
    # Match only lines like CRT-xxxx: name
    match = re.match(r"^(CRT-\d+):\s+([\w\d_-]+)$", line.strip())
    if match:
        crt, dev = match.groups()
        result_list.append([crt, dev])

print(result_list)



[]


In [2]:
'''now assign the CRT to the developer we got from LLM'''
import requests
import json

# Jira instance URL
JIRA_URL = "https://ca-il-jira.il.cyber-ark.com:8443/rest/api/2/issue"


BEARER_TOKEN = "xGkGLtKttyM5Gnr57XLyPYMg2k1RwpnTyBiR8P"  


assignments = [
    ["ibulochkina", "CRT-8510"],
    ["jdoe", "CRT-8511"]
    
]

def assign_jira_issue(crt_number, developer_name):
    """

    """
    url = f"{JIRA_URL}/{crt_number}/assignee"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {BEARER_TOKEN}"
    }
    payload = {
        "name": developer_name
    }
    
    try:
        response = requests.put(
            url,
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code == 204:
            print(f"Successfully assigned {crt_number} to {developer_name}")
        else:
            print(f"Failed to assign {crt_number} to {developer_name}: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        print(f"Error assigning {crt_number} to {developer_name}: {str(e)}")

In [19]:
for crt, dev in result_list:
    assign_jira_issue(crt, dev)